# Test pointcloud

In [1]:
import pytest
import numpy as np
from firedrake import *
from firedrake import parloops

from peval.pointcloud import PointCloud


import matplotlib.pyplot as plt

# logging.set_level(logging.DEBUG)

# disable the warning message of loopy
parloops.LOOPY_USE_LANGUAGE_VERSION_2018_2 = parloops.loopy.version.LOOPY_USE_LANGUAGE_VERSION_2018_2

m = UnitSquareMesh(16, 16, quadrilateral=True)
x = SpatialCoordinate(m)

Vv = VectorFunctionSpace(m, 'CG', 5)
fv = Function(Vv)
fv.interpolate(as_vector([x[0]**2, x[1]**2]))


# m.topology_dm.view()
m_ref = UnitSquareMesh(32, 32, quadrilateral=True)
Vv_ref = VectorFunctionSpace(m_ref, 'CG', 5)


coords_ref = Function(Vv_ref)
coords_ref.interpolate(m_ref.coordinates)

tol = 1e-14
coords_ref = coords_ref.dat.data
coords_ref[coords_ref[:, 1] > 1 - tol, 1] = 1 - tol
coords_ref[coords_ref[:, 0] > 1 - tol, 0] = 1 - tol

vm = VertexOnlyMesh(m, coords_ref)
W = VectorFunctionSpace(vm, "DG", 0)
f = interpolate(fv, W)

pc = PointCloud(m, coords_ref)
v = pc.evaluate(fv)
fv_ref = Function(Vv_ref)
fv_ref.dat.data[:] = v

# tricontourf(fv)
# tricontourf(fv_ref)



firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


In [2]:
from firedrake.utility_meshes import UnitSquareMesh
from firedrake.functionspace import FunctionSpace, VectorFunctionSpace
from firedrake.function import Function

from ufl.geometry import SpatialCoordinate 

def get_fun(n, degree=None, fun=None):
    degree = degree or 2
    fun = fun or (lambda x: x[0]**4 + x[1]**4)

    quadrilateral=False
    m = UnitSquareMesh(n, n, quadrilateral=quadrilateral)
    V = FunctionSpace(m, 'CG', degree)
    f = Function(V)
    x = SpatialCoordinate(m)
    f.interpolate(fun(x))
    
    return f

def get_fun_kvm(n, degree=None, fun=None):
    degree = degree or 2
    fun = fun or (lambda x: x[0]**4 + x[1]**4)

    quadrilateral=False
    m = UnitSquareMesh(n, n, quadrilateral=quadrilateral)
    V = FunctionSpace(m, 'KMV', degree)
    f = Function(V)
    x = SpatialCoordinate(m)
    f.interpolate(fun(x))
    
    return f

In [34]:
from firedrake.functionspace import FunctionSpace, VectorFunctionSpace
from firedrake.function import Function
from firedrake.norms import errornorm
from firedrake.mesh import VertexOnlyMesh

from peval import PointCloud

def get_nodes_coords(fun):
    ''' Get the coords of the dofs of Function fun or Space'''

    mesh = fun.ufl_domain()
    element = fun.ufl_element()
    
    assert(element.family() == 'Lagrange')

    degree = element.degree()
    if degree == 1:
        coords = mesh.coordinates.dat.data
    else:
        C = VectorFunctionSpace(mesh, 'CG', degree)
        interp_coordinates = Function(C)
        interp_coordinates.interpolate(mesh.coordinates)
        coords = interp_coordinates.dat.data

    return coords

def get_common_function_space(u, u_ref):
    '''Get the CG space for compute errors of u and u_ref
    '''
    element = u.ufl_element()
    element_ref = u_ref.ufl_element()
    assert(element == element_ref)
    
    if element.family() == 'Lagrange':
        V_inter = u_ref.ufl_function_space()
        return V_inter, False
    
    mesh = u_ref.ufl_domain()
    V = u_ref.ufl_function_space()
    degree = element_ref.degree() + 2 # may be just add 2?
    if V.rank == 1:
        V_inter = VectorFunctionSpace(mesh, 'CG', degree=degree, dim=element_ref.value_size())
    elif V.rank == 0:
        V_inter = FunctionSpace(mesh, 'CG', degree=degree)
    else:
        raise NotImplementedError('Do not support space with rank > 1')
        
    return V_inter, True

def get_pointclouds(u, u_ref, tolerance):
    V_inter, need_inter_u_ref = get_common_function_space(u, u_ref)
    coords = get_nodes_coords(V_inter)
    u_inter = Function(V_inter)
    
    pc = PointCloud(u.ufl_domain(), coords, tolerance=tolerance)
    
    if need_inter_u_ref:
        pc_ref = PointCloud(u_ref.ufl_domain(), coords, tolerance=tolerance)
        u_ref_inter = Function(V_inter)
    else:
        pc_ref = None
        u_ref_inter = None
    
    return pc, pc_ref, u_inter, u_ref_inter


def compute_errors_pc(u, u_ref, assistant=None, tolerance=None, norm_type="L2"):
    if assistant is None:
        pc, pc_ref, u_inter, u_ref_inter = get_pointclouds(u, u_ref, tolerance)
    else:
        pc, pc_ref, u_inter, u_ref_inter = assistant
        
    u_inter.dat.data[:] = pc.evaluate(u)

    if pc_ref is not None:
        u_ref_inter.dat.data[:] = pc_ref.evaluate(u_ref)
    else:
        u_ref_inter = u_ref
        
    return errornorm(u_ref_inter, u_inter, norm_type=norm_type)

In [31]:
n = 5
fs_ = [get_fun(2**(_+1)) for _ in range(n)]
fs = [get_fun_kvm(2**(_+1)) for _ in range(n)]

errs = [compute_errors_pc(fs_[_], fs_[_+1], tolerance=1e-10) for _ in range(n-1)]
errs

errs2 = [compute_errors_pc(fs[_], fs[_+1], tolerance=1e-10) for _ in range(n-1)]
errs2

In [35]:
f.comm.b

0

In [36]:
f.comm.size

1

In [29]:
%debug

> <ipython-input-28-6a7a3264e47e>(1)<module>()
----> 1 point



ipdb>  up


*** Oldest frame


ipdb>  up


*** Oldest frame


ipdb>  exit


# Test VertexOnlyMesh

In [14]:
from firedrake import *
# logging.set_level(logging.DEBUG)
def test_vom(n1=8, n2=16, quadrilateral=False):
    m = UnitSquareMesh(n1, n1, quadrilateral=quadrilateral)
    m_ref = UnitSquareMesh(n2, n2, quadrilateral=quadrilateral)
    coords_ref = m_ref.coordinates.dat.data_ro.copy()
    
    # make sure they are in the domain
#     tol = 1e-14
#     coords_ref[coords_ref[:, 1] > 1 - tol, 1] = 1 - tol
#     coords_ref[coords_ref[:, 0] > 1 - tol, 0] = 1 - tol
    
    vm = VertexOnlyMesh(m, coords_ref)
    W = FunctionSpace(vm, "DG", 0)
    print('\tThere are %d points'%len(coords_ref))
    print('\tThere are %d dofs in space W'%W.dof_count)
    return m, m_ref

print('Test with quadrilateral=False')
test_vom(n1=8, n2=16, quadrilateral=False)
print('Test with quadrilateral=True')
test_vom(n1=8, n2=16, quadrilateral=True)
# print('Test with larger n1 and quadrilateral=False')
# test_vom(n1=10, n2=16, quadrilateral=False)

Test with quadrilateral=False
	There are 289 points
	There are 289 dofs in space W
Test with quadrilateral=True
	There are 289 points
	There are 256 dofs in space W


(Mesh(VectorElement(FiniteElement('Q', quadrilateral, 1), dim=2), 83),
 Mesh(VectorElement(FiniteElement('Q', quadrilateral, 1), dim=2), 84))